In [1]:
!pip install numpy pandas matplotlib tensorflow opencv-python albumentations scikit-learn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.8 MB ? eta -:--:--
    --------------------------------------- 0.3/15.8 MB ? eta -:--:--
    -------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import cv2
import albumentations as A
from sklearn.model_selection import train_test_split
import os

C:\Users\DHARSAN k\AppData\Roaming\Python\Python311\site-packages\albumentations\check_version.py:107: UserWarning: Error fetching version info <urlopen error [Errno 11001] getaddrinfo failed>
  data = fetch_version_info()


In [8]:
# Load images and masks from dataset path
def load_images_and_masks(image_path, mask_path):
    images = []
    masks = []
    
    image_files = sorted(os.listdir(image_path))
    mask_files = sorted(os.listdir(mask_path))
    
    for img_file, mask_file in zip(image_files, mask_files):
        img = cv2.imread(os.path.join(image_path, img_file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256, 256))  # Resize to 256x256
        img = img / 255.0  # Normalize
        
        mask = cv2.imread(os.path.join(mask_path, mask_file), cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (256, 256))
        mask = mask / 255.0  # Normalize to 0-1
        
        images.append(img)
        masks.append(mask)
    
    return np.array(images), np.array(masks).reshape(-1, 256, 256, 1)

# Provide correct dataset paths
image_path = "REFUGE/Images_Square"
mask_path = "REFUGE/Masks_Square"

images, masks = load_images_and_masks(image_path, mask_path)

# Split dataset into train, validation, and test sets (80-10-10)
X_train, X_temp, y_train, y_temp = train_test_split(images, masks, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}, Test samples: {len(X_test)}")

Training samples: 960, Validation samples: 120, Test samples: 120


In [9]:
# Define augmentation pipeline using Albumentations
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=20, p=0.5)
])

def augment_data(images, masks):
    augmented_images = []
    augmented_masks = []
    
    for img, mask in zip(images, masks):
        augmented = augment(image=img, mask=mask)
        augmented_images.append(augmented['image'])
        augmented_masks.append(augmented['mask'])
    
    return np.array(augmented_images), np.array(augmented_masks)

# Augment training data
X_train_aug, y_train_aug = augment_data(X_train, y_train)

# Combine original and augmented data
X_train = np.concatenate((X_train, X_train_aug))
y_train = np.concatenate((y_train, y_train_aug))

print(f"New training size after augmentation: {len(X_train)}")

New training size after augmentation: 1920


In [10]:
# Define Residual Block
def residual_block(x, filters):
    res = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(x)
    res = layers.BatchNormalization()(res)
    res = layers.Conv2D(filters, (3, 3), padding='same', activation='relu')(res)
    res = layers.BatchNormalization()(res)
    
    shortcut = layers.Conv2D(filters, (1, 1), padding='same')(x)
    out = layers.Add()([res, shortcut])
    return layers.Activation('relu')(out)

# Define Attention Gate
def attention_gate(x, g, filters):
    x = layers.Conv2D(filters, (1, 1), padding='same')(x)
    g = layers.Conv2D(filters, (1, 1), padding='same')(g)
    
    add = layers.Add()([x, g])
    activation = layers.Activation('relu')(add)
    
    attention = layers.Conv2D(1, (1, 1), activation='sigmoid')(activation)
    return layers.Multiply()([x, attention])

# Define U-Net with Attention and Residual Blocks
def improved_unet(input_shape=(256, 256, 3)):
    inputs = layers.Input(input_shape)
    
    # Encoder
    c1 = residual_block(inputs, 64)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    
    c2 = residual_block(p1, 128)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    
    c3 = residual_block(p2, 256)
    p3 = layers.MaxPooling2D((2, 2))(c3)
    
    c4 = residual_block(p3, 512)
    p4 = layers.MaxPooling2D((2, 2))(c4)
    
    # Bottleneck
    c5 = residual_block(p4, 1024)
    
    # Decoder
    u6 = layers.UpSampling2D((2, 2))(c5)
    a6 = attention_gate(c4, u6, 512)
    c6 = residual_block(layers.Concatenate()([u6, a6]), 512)
    
    u7 = layers.UpSampling2D((2, 2))(c6)
    a7 = attention_gate(c3, u7, 256)
    c7 = residual_block(layers.Concatenate()([u7, a7]), 256)
    
    u8 = layers.UpSampling2D((2, 2))(c7)
    a8 = attention_gate(c2, u8, 128)
    c8 = residual_block(layers.Concatenate()([u8, a8]), 128)
    
    u9 = layers.UpSampling2D((2, 2))(c8)
    a9 = attention_gate(c1, u9, 64)
    c9 = residual_block(layers.Concatenate()([u9, a9]), 64)
    
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
    
    return models.Model(inputs, outputs)

model = improved_unet()
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",
              metrics=["accuracy", tf.keras.metrics.MeanIoU(num_classes=2)])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 256, 256, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d (Conv2D)               │ (None, 256, 256, 64)      │           1,792 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization           │ (None, 256, 256, 64)      │             256 │ conv2d[0][0]               │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_1 (Conv2D)             │ (None, 256, 256, 64)      │          36,928 │ batch_normalization[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_1         │ (None, 256, 256, 64)      │             256 │ conv2d_1[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_2 (Conv2D)             │ (None, 256, 256, 64)      │             256 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add (Add)                     │ (None, 256, 256, 64)      │               0 │ batch_normalization_1[0][… │
│                               │                           │                 │ conv2d_2[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ activation (Activation)       │ (None, 256, 256, 64)      │               0 │ add[0][0]                  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ max_pooling2d (MaxPooling2D)  │ (None, 128, 128, 64)      │               0 │ activation[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_3 (Conv2D)             │ (None, 128, 128, 128)     │          73,856 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_2         │ (None, 128, 128, 128)     │             512 │ conv2d_3[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_4 (Conv2D)             │ (None, 128, 128, 128)     │         147,584 │ batch_normalization_2[0][… │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ batch_normalization_3         │ (None, 128, 128, 128)     │             512 │ conv2d_4[0][0]             │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_5 (Conv2D)             │ (None, 128, 128, 128)     │           8,320 │ max_pooling2d[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ add_1 (Add)                   │ (None, 128, 128, 128)     │               

 Total params: 34,193,797 (130.44 MB)

 Trainable params: 34,182,021 (130.39 MB)

 Non-trainable params: 11,776 (46.00 KB)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

# Define Early Stopping Callback
early_stop = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=5,          # Stop training if val_loss does not improve for 5 epochs
    restore_best_weights=True  # Restore the best model state
)

# Train the model with Early Stopping
with tf.device('/GPU:0'):  # Use GPU for training
    history = model.fit(X_train, y_train, 
                        validation_data=(X_val, y_val), 
                        epochs=10, batch_size=8,  # Changed epochs to 10
                        callbacks=[early_stop])  # Include early stopping

# Save the model
model.save("improved_unet_glaucoma.h5")

Epoch 1/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2831s 12s/step - accuracy: 0.9146 - loss: 0.1484 - mean_io_u: 0.5000 - val_accuracy: 0.9838 - val_loss: 9.7427e-04 - val_mean_io_u: 1.0000
Epoch 2/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2805s 12s/step - accuracy: 0.9835 - loss: 6.3433e-04 - mean_io_u: 1.0000 - val_accuracy: 0.9838 - val_loss: 7.6876e-04 - val_mean_io_u: 1.0000
Epoch 3/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2812s 12s/step - accuracy: 0.9836 - loss: 5.7697e-04 - mean_io_u: 1.0000 - val_accuracy: 0.9838 - val_loss: 5.6125e-04 - val_mean_io_u: 1.0000
Epoch 4/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2815s 12s/step - accuracy: 0.9835 - loss: 5.6733e-04 - mean_io_u: 1.0000 - val_accuracy: 0.9838 - val_loss: 5.4067e-04 - val_mean_io_u: 1.0000
Epoch 5/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2818s 12s/step - accuracy: 0.9835 - loss: 5.5069e-04 - mean_io_u: 1.0000 - val_accuracy: 0.9838 - val_loss: 5.3411e-04 - val_mean_io_u: 1.0000
Epoch 6/10
240/240 ━━━━━━━━━━━━━━━━━━━━ 2820s 12s/step - accuracy: 0.9836 - loss: 5.4357e